In [3]:
import pandas as pd
from urllib.request import urlopen, Request
from bs4 import BeautifulSoup as bs
import time
from parse_str import *
import requests
import json
from datetime import datetime

In [4]:
# API URL
symbol = 'rpar'
url = f"https://api.stockanalysis.com/api/symbol/e/{symbol}/overview"

# API 요청 보내기
response = requests.get(url)
data = response.json()

# ETF의 주요 정보 추출 및 출력
etf_data = data.get("data", {})
data = etf_data.get("infoTable")
data

[['Asset Class', 'Alternatives'],
 ['Category', 'Moderately Conservative Allocation'],
 ['Region', 'North America'],
 ['Stock Exchange', 'NYSEARCA'],
 ['Ticker Symbol', 'RPAR'],
 ['Provider', 'Tidal']]

In [7]:
# etf_data.keys()
# 분할해서 만든다: 비효율적이어도 디버그 하기 쉽다

def get_etf_profile(symbol: str):
    # API URL
    url = f"https://api.stockanalysis.com/api/symbol/e/{symbol}/overview"

    # API 요청 보내기
    response = requests.get(url)
    response = response.json()

    etf_data = response.get("data", {})
    data = etf_data.get("infoTable")
    if not etf_data:
        print(f"Empty Response: {symbol}")
        return None
    
    data_dict = {item[0]: item[1] for item in data}
    df = pd.DataFrame([data_dict])
    
    # 컬럼명 수정
    df.rename(columns={
        "Asset Class": "asset_class",
        "Category": "category",
        "Region": "region",
        "Stock Exchange": "stock_exchange",
        "Ticker Symbol": "symbol",
        "Provider": "fund_family",
        "Index Tracked": "index_tracked"
    }, inplace=True)

    # 결측치 삽입
    if "category" not in df.columns:
        df["category"] = "n/a"
    if "region" not in df.columns:
        df["region"] = "n/a"
    if "index_tracked" not in df.columns:
        df["index_tracked"] = "n/a"

    # 컬럼 순서 정렬
    df = df[[
       "symbol", "asset_class", "category", "region", "stock_exchange", "fund_family", "index_tracked"
    ]]

    return df

# display(get_etf_profile("xlv"))
display(get_etf_profile("wfh"))
display(get_etf_profile("flcb"))
display(get_etf_profile("flaab"))

,symbol,asset_class,category,region,stock_exchange,fund_family,index_tracked
0,WFH,Equity,Technology,n/a,NYSEARCA,Direxion,Solactive Remote Work Index


,symbol,asset_class,category,region,stock_exchange,fund_family,index_tracked
0,FLCB,Fixed Income,Intermediate Core Bond,North America,NYSEARCA,Franklin Templeton,n/a


Empty Response: flaab


None